In [ ]:
import cobra
from tqdm import tqdm
import csv
import re
import urllib
import pickle


### DNA features 

In [ ]:
def get_features_10(x):
    Cmodel = cobra.io.read_sbml_model('/data1/xpgeng/P1/model/'+ x)
    model_id = x[:-4]
    Cgeneid = []
    for i in Cmodel.genes:
        Cgeneid.append(i.id)
        #Cgeneid = Cgeneid[:2]
    Cseqs = {}
    C_gene_mop = {}
    C_gene_asso_reac_count = {}
    C_protein_seqs = {}
    
    for i in tqdm(Cgeneid):
        l = []
        html = urllib.request.urlopen('http://bigg.ucsd.edu/models/'+ x[:-4] +'/genes/'+ i)
        b = str(html.read()).split('\\n')
        for j in range(len(b)):
            if 'DNA Sequence' in b[j]:        
                Cseqs[i] = re.findall(r'<p class="sequence">(.+?)</p>',b[j+1])[0]
            if '<h4>Strand:' in b[j]:        
                C_gene_mop[i] = re.findall(r'<p>(.+?)</p>',b[j+1])[0]
            if 'Protein Sequence' in b[j]:        
                C_protein_seqs[i] = re.findall(r'<p class="sequence">(.+?)</p>',b[j+1])[0]
            if f'href="/models/{model_id}/reactions' in b[j]:
                l.append(re.findall(r'/reactions/(.+?)">',b[j])[0])
        C_gene_asso_reac_count[i] = l
    
    C_ignored_genes = []
    for i in Cgeneid:
        if i not in Cseqs and i not in C_protein_seqs:
            C_ignored_genes.append(i)
        elif i not in C_protein_seqs and (len(Cseqs[i]) % 3 != 0 or len(Cseqs[i]) == 0):
            C_ignored_genes.append(i)
            
    for i in C_ignored_genes:
        Cgeneid.remove(i)
    
    # GC ratio
    C_GC = {}
    for i in Cgeneid:
        gc = 0
        for j in Cseqs[i]:
            if j in ['G', 'C']:
                gc += 1
        C_GC[i] = round(gc / len(Cseqs[i]), 3)
    print(len(C_GC)) 

    # associated reaction number
    C_asso_rea_num = {}
    for i in Cmodel.genes:
        if i.id in C_ignored_genes: # exclude biomass function
            continue
        C_asso_rea_num[i.id] = len(i.reactions)
    print(len(C_asso_rea_num))

    # gene length
    C_length = {}
    for i in Cgeneid:
        C_length[i] = len(Cseqs[i])
    print(len(C_length))
    
    
    C_rea_asso_enzy_count = {}
    for i in tqdm(Cmodel.reactions, desc='crawling reactions info'):
        rea_id = i.id if '_copy' not in i.id else i.id.split("_copy")[0] # exclude copy reactions
        html = urllib.request.urlopen(f'http://bigg.ucsd.edu/models/{model_id}/reactions/' + rea_id)
        b = str(html.read()).split('\\n')
        enzylis = []
        for j in range(len(b)):
            if '//identifiers.org/ec-code' in b[j]:        
                enzylis.append( re.findall(r'identifiers.org/ec-code/(.+?)"',b[j])[0].split(".")[0] )
        C_rea_asso_enzy_count[rea_id] = enzylis
        
    C_gene_asso_enzy_count = []
    for i in Cgeneid:
        l = []
        for j in C_gene_asso_reac_count[i]:
            try:        
                l += C_rea_asso_enzy_count[j]
            except:
                print(j)
        C_gene_asso_enzy_count.append(l)
    print(len(C_gene_asso_enzy_count))

    for i in range(len(C_gene_asso_enzy_count)):
        try:
            C_gene_asso_enzy_count[i] = list(set(C_gene_asso_enzy_count[i]))
        except:
            print(C_gene_asso_enzy_count[i])

    for i in range(len(C_gene_asso_enzy_count)):
        for j in range(len(C_gene_asso_enzy_count[i])):
            C_gene_asso_enzy_count[i][j] = int(C_gene_asso_enzy_count[i][j])

    C_gene_asso_enzy = []
    for i in C_gene_asso_enzy_count:
        b = [1 if (k+1) in i else 0 for k in range(6)]
        C_gene_asso_enzy.append(b)
    print(len(C_gene_asso_enzy))

    Cgc = []
    for i in Cgeneid:
        Cgc.append(C_GC[i])
    print(len(Cgc))

    Casn = []
    for i in Cgeneid:
        Casn.append(round(C_asso_rea_num[i]/10,3))
    print(len(Casn))

    Clen = []
    for i in Cgeneid:
        Clen.append(round(C_length[i]/100,3))
    print(len(Clen))

    Cmop = []
    for i in Cgeneid:
        if C_gene_mop[i] == 'Minus':
            Cmop.append(-1)
        elif C_gene_mop[i] == 'Plus':
            Cmop.append(1)
    print(len(Cmop))
    
    dic = {}
    for i in range(len(Cgeneid)):
        dic[Cgeneid[i]] = [Cmop[i], Clen[i], Casn[i], Cgc[i]] + C_gene_asso_enzy[i]

    
    with open(x[:-4]+'_10_features.pkl', 'wb') as f:
        pickle.dump(dic, f)

### Protein embedding 

In [ ]:
# first change path as the protein file cause path is different here

from bio_embeddings.embed import ProtTransBertBFDEmbedder
from Bio import SeqIO

def data_prepare_embedding(model_id, middle_path):
    
    sequences = []
    for record in SeqIO.parse(os.path.join(middle_path, f'{model_id}_protein_sequences.txt'), "fasta"):
        sequences.append(record)

    embedder = ProtTransBertBFDEmbedder()

    embeddings = embedder.embed_many([str(s.seq) for s in sequences])

    # `embed_many` returns a generator.
    # We want to keep both RAW embeddings and reduced embeddings in memory.
    # To do so, we simply turn the generator into a list!
    # (this will start embedding the sequences!)

    embeddings = list(embeddings)

    reduced_embeddings = {s.id: list(ProtTransBertBFDEmbedder.reduce_per_protein(e)) for s,e in zip(sequences, embeddings)}

    with open(os.path.join('/data1/xpgeng/P1/raw_data', f'{model_id}_protein_embedding.pkl'), "wb") as f:
        pickle.dump(reduced_embeddings, f)
        
data_prepare_embedding('iML1515', '/data1/xpgeng/P1/raw_data/model_seq')